In [5]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)  # initialize

memory = ConversationSummaryMemory(llm=llm)  # cost !


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Jimin, I live in South Korea", "Wow that is so cool!")

In [6]:
add_message("South Korea is so comfortable", "I wish I could go!!!")

In [8]:
get_history()

{'history': 'The human introduces himself as Jimin and mentions that he lives in South Korea, to which the AI responds with enthusiasm, expressing that it is cool. Jimin then describes South Korea as comfortable, and the AI expresses a wish to visit.'}